In [1]:
import sys
from pathlib import Path
src_dir = Path.cwd().parent
sys.path.append(str(src_dir))
from utilits.utilits import *
import torch
import os
from tqdm.auto import tqdm
import evaluate
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from models.NoiseAwareLoss import (
    NormalizedCrossEntropy,
    ReverseCrossEntropy,
    NCEandRCE,
    MeanAbsoluteError,
    NCEandMAE
)
from collections import defaultdict
from transformers import get_scheduler , AutoConfig
from collections import defaultdict

In [2]:
from datasets import load_dataset
data_files = {"train" : "train.json" , "test" : "eval.json"}
train_data = load_dataset("json" , data_files=data_files)

Using custom data configuration default-dbdc4e096e283662
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-dbdc4e096e283662/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)
model.to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [4]:
def tokenize( batch):
        """
        Tokenizes input text and returns a dictionary of tokenized inputs.

        Parameters:
        - batch: a dictionary representing a batch of input data

        Returns:
        - a dictionary containing the tokenized inputs

        """
        return tokenizer(batch["tweets"], padding="max_length", truncation=True)
        
tokenized_dataset = train_data.map(tokenize , batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["tweets"])
tokenized_dataset.set_format('torch')

train_dataloader = DataLoader(tokenized_dataset["train"] , shuffle=True , batch_size=16)
eval_dataloader = DataLoader(tokenized_dataset["test"] , shuffle=True , batch_size=16)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-dbdc4e096e283662/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-cb2109ffcfb3e32c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-dbdc4e096e283662/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-6a11d0e3dee3d889.arrow


In [5]:
import evaluate

num_epochs= 5
learning_rate = 2e-5
weight_decay = 0.001

num_training_steps =  num_epochs * len(train_dataloader)

losses = defaultdict(list)
progress_bar = tqdm(range(num_training_steps), colour="green")
#initialize the model

#accuracy
accuracy_metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
#Confusion Metrix 
preds_labels = defaultdict()
optimizer = AdamW(model.parameters(), lr=learning_rate)
LR_Schaduler =get_scheduler(name='linear' ,
                            optimizer= optimizer,
                            num_warmup_steps = 0,
                            num_training_steps= num_training_steps)
metric = evaluate.load("accuracy")

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss_fct = NCEandMAE(1,1,2)
        #loss = loss_fct(outputs.logits.view(-1, 2), batch["labels"].view(-1))
        loss = loss_fct(outputs.logits , batch["labels"])
        loss.backward()
        optimizer.step()
        LR_Schaduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    print(loss)
    
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()


  0%|          | 0/2980 [00:00<?, ?it/s]

  0%|          | 0/2980 [00:00<?, ?it/s]

tensor(0.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3334, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3333, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3333, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1667, device='cuda:0', grad_fn=<AddBackward0>)


{'accuracy': 0.8217281879194631}

In [7]:
olid_dataset = prepare_data("../dataset/")
olid_dataset

{'train': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 13240
 }),
 'test': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 3887
 })}

In [8]:
olid_dataset_emd = olid_dataset['test'].map(tokenize ,batched=True)
olid_dataset_emd_test = olid_dataset_emd.remove_columns(["tweets" , "labels_name"])
olid_dataset_emd_test.set_format('torch')

olid_dataset_emd_test_loader = DataLoader(olid_dataset_emd_test , shuffle=True , batch_size=16)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [9]:
model.eval()
for batch in olid_dataset_emd_test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

{'accuracy': 0.7221507589400566}